In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import random
import time

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
IMAGES_FOLDER = '/home/dan/datasets/COCO/images/val2017/'

# Load the model

In [ ]:
detector = Detector(MODEL_PATH, visible_device_list='1')

# Get an image

In [ ]:
names = os.listdir(IMAGES_FOLDER)

In [ ]:
n = random.choice(names)
path = os.path.join(IMAGES_FOLDER, n)

image = Image.open(path)
image = image.resize((640, 640))
image

# Predict

In [ ]:
image_array = np.array(image)
outputs = detector(image_array, score_threshold=0.0001)

# Show detections

In [ ]:
EDGES = [
    (0, 1), (0, 2),
    (1, 3), (2, 4),
    (5, 7), (7, 9), (6, 8), (8, 10),
    (11, 13), (13, 15), (12, 14), (14, 16),
    (3, 5), (4, 6),
    (5, 11), (6, 12)
]

In [ ]:
outputs['keypoint_scores']

In [ ]:
def draw_everything(image, outputs):

    image_copy = image.copy()
    image_copy.putalpha(255)
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image_copy.size
    
    scaler = np.array([height, width, height, width])
    boxes = scaler * outputs['boxes']

    for i, box in enumerate(boxes):
        
        ymin, xmin, ymax, xmax = box
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline='red')
       
        keypoints = outputs['keypoint_positions'][i]
        keypoints = keypoints[:, [1, 0]].copy()
        keypoints *= np.array([xmax - xmin, ymax - ymin])
        keypoints += np.array([xmin, ymin])
        visibility = np.ones([17, 1], dtype=np.float32)
        keypoints = np.concatenate([keypoints, visibility], axis=1)

        for (p, q) in EDGES:

            x1, y1, v1 = keypoints[p]
            x2, y2, v2 = keypoints[q]

            both_visible = v1 > 0 and v2 > 0
            if both_visible:
                draw.line([(x1, y1), (x2, y2)])

        for j in range(17):
            x, y, v = keypoints[j]
            if v > 0:
                s = 2
                draw.ellipse([
                    (x - s, y - s),
                    (x + s, y + s)
                ], fill='red')

    return image_copy

In [ ]:
draw_everything(image, outputs)

In [ ]:
segmentation_masks = outputs['segmentation_masks'].copy()
m = segmentation_masks.min()
M = segmentation_masks.max()
segmentation_masks = 255.0 * (segmentation_masks - m)/(M - m)
segmentation_masks = Image.fromarray(segmentation_masks.astype('uint8'))
segmentation_masks

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    result = detector(image_array, score_threshold=0.25)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())

# Show heatmaps 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

cmap = plt.cm.get_cmap('autumn')
new_cmap = cmap(np.arange(cmap.N))
new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
cmap = ListedColormap(new_cmap)  # create new colormap

In [ ]:
ORDER = {
    0: 'nose',
    1: 'left eye', 2: 'right eye',
    3: 'left ear', 4: 'right ear',
    5: 'left shoulder', 6: 'right shoulder',
    7: 'left elbow', 8: 'right elbow',
    9: 'left wrist', 10: 'right wrist',
    11: 'left hip', 12: 'right hip',
    13: 'left knee', 14: 'right knee',
    15: 'left ankle', 16: 'right ankle'
}


def plot_maps(image, heatmaps, segmentation_mask):
    """
    Arguments:
        image: a float numpy array with shape [h, w, 3].
        heatmaps: a float numpy array with shape [h / 4, w / 4, 17].
        segmentation_mask: a float numpy array with shape [h / 4, w / 4].
        loss_mask: a float numpy array with shape [h / 4, w / 4].
    """

    h, w, _ = image.shape
    h, w = (h // 2), (w // 2)
    background = Image.new('RGBA', (w, h * 18), (255, 255, 255, 255))
    draw = ImageDraw.Draw(background, 'RGBA')
    
    image = Image.fromarray(image)
    image = image.resize((w, h), Image.LANCZOS)
    image.putalpha(255)

    heatmaps = (255 * cmap(heatmaps)).astype('uint8')
    # it has shape [h, w, 17, 4]
    
    heats = []
    for j, name in ORDER.items():

        heat = Image.fromarray(heatmaps[:, :, j])
        heat = heat.resize((w, h), Image.LANCZOS)
        heat = Image.alpha_composite(image, heat)
        background.paste(heat, (0, j * h))
        draw.text((0, j * h), name, fill='red')
        
    def draw_mask(mask):
        mask = np.clip(mask, 0.0, 1.0)
        mask = (255 * mask).astype('uint8')
        mask = Image.fromarray(mask)
        mask = mask.resize((w, h), Image.LANCZOS).convert('RGB')
        mask.putalpha(mask.convert('L'))
        mask = Image.alpha_composite(image, mask)
        return mask
    
    mask = draw_mask(segmentation_mask)
    background.paste(mask, (0, 17 * h))
    draw.text((0, 17 * h), 'segmentation mask', fill='red')

    return background

In [ ]:
h = outputs['keypoint_heatmaps']
m = h.min(0).min(0)
M = h.max(0).max(0)
h = (h - m)/(M - m)

plot_maps(image_array, h, outputs['segmentation_masks'])